In [1]:
import pywikibot
import time
import re

In [2]:
site = pywikibot.Site('fr', 'wiktionary')
print(site)

wiktionary:fr


In [3]:
# This parameter represents the character or set of consecutive characters 
# to be replaced
old_symbol = 'õ' # regular 'g' letter 

# This parameter represents the character or set of consecutive characters 
# that will replace the old_symbol
new_symbol = 'ɔ̃'

In [4]:
words = [
    'Aumont',
    'Avignon-lès-Saint-Claude',
    'Caf’-Con’',
    'Léontine',
    'Léontines',
    'Montagnac',
    'Montmarlon',
    'Valparaiso',
    'autocontradiction',
    'autocontradictions',
    'besson',
    'casquette à pont',
    'casquetttes à pont',
    'charcher son pain',
    'compte prorata',
    'comptes proratas',
    'continuo',
    'continuos',
    'crayon à mine',
    'de bon hait',
    'dombra',
    'dombras',
    'fantons',
    'maisonne',
    'mon Guieu faut-i',
    'poiçon',
    'poiçons',
    'pomon',
    'pomons',
    'potron-jacquet',
    'quatrième dimension',
    'rhombicuboctaèdre',
    'rodonticide',
    'rodonticides',
    'rodontologie',
    'rodontologies',
    'rodontologique',
    'rodontologiques',
    'ronflote',
    'sconses',
    'sur l’ongle',
    'union libre',
    'volontourisme',
    'volontourismes',
]

In [5]:
MODELES_SIMPLES = [
    'fr-accord-cons',
    'fr-accord-en',
    'fr-accord-on',
    'fr-accord-rég',
    'fr-rég', 
    'fr-inv', 
]
MODELES_DOUBLES = [
    'fr-accord-al',
    'fr-accord-eur',
    'fr-accord-eux',
    'fr-accord-mf',
    'fr-accord-mf-al',
]
MODELES_TROIS = [
    'fr-accord-mf',
]
MODELES_TO_IGNORE = [
    'fr-verbe-flexion',
]
MODELES = MODELES_SIMPLES + MODELES_DOUBLES + MODELES_TROIS

In [6]:
ALLOWED_PHONEMES = [
        # Voyelles
        'i','e','ɛ','a','ɑ','ɔ','o','u','y','ø','œ','ə','ɛ̃','ɑ̃','ɔ̃','œ̃',
        # Semi-consonnes
        'j','w','ɥ', 
        # Consonnes
        'p','t','k','b','d','ɡ','f','s','ʃ','v','z','ʒ','l','ʁ','m','n','ɲ','h','ŋ',
        # Addĩitional ones. They should not be there (most of them are user errors), 
        # but if not taken into account, they lead to more words being discarded
        '(',')','-',' ','ˈ', '.', '\ ̃',
        #'ɡ','ǝ', 'r', 'ã', 'ʏ', 'ẽ', 'õ', 
        '‿', # pitches
]

In [7]:
def skip_word(symbol, page, word):
    if symbol == 'ẽ':
        if 'CA' in page or 'QC' in page:
            print('skip_word: symbol %s probably good in %s' % (symbol, word))
            return True
    return False

In [8]:
def check_phonemes(string, verbose=False):
    nb_chars = len(string)
    
    if nb_chars == 0:
        return('check_phonemes: received empty string !!!')
    
    nb_chars_allowed = 0
    probably = True
    for symbol in string:
        if symbol in ALLOWED_PHONEMES:
            nb_chars_allowed += 1
        else:
            if verbose:
                print('check_phonemes: in %s, found %c (ascii=%d)' % (string, symbol, ord(symbol)))
    pctg = int (nb_chars_allowed/nb_chars*100)
    if verbose:
        print('check_phonemes(%s) : %d%% of good phonemes' % (string, pctg))
    return pctg

In [9]:
def replace_pronunciations_in_ligne_de_forme(text, pronunciations, old_symbol, new_symbol, verbose=False):
    nb_replacements = 0
    for pron in pronunciations:
        if verbose:
            print('found:', pron)
        old_string = pron[0] + pron[1] + pron[2]
        pron_new = pron[1].replace(old_symbol, new_symbol)
        new_string = pron[0] + pron_new + pron[2]
        print('replacing : %s\nwith      : %s' % (old_string, new_string))    
        text = text.replace(old_string, new_string)
        if new_string != old_string:
            nb_replacements += 1
    print('nb_replacements in ldf:%d', nb_replacements)
    return text

In [10]:
def replace_pronunciations_in_model(text, pronunciations, old_symbol, new_symbol, verbose=False):
    nb_replacements = 0
    for pron in pronunciations:
        if verbose:
            print('found:', pron)
        old_string = pron[0] + pron[1] + pron[2]
        if verbose:
            print('old_string:%s ' % old_string)
        
        # extract modele from string like '{{fr-rég|'
        modele = pron[0][2:-1]
        if verbose:
            print('modele:%s' % modele)
        if modele in MODELES_TO_IGNORE:
            print('modele with no pronunciation:%s !!!' % modele)
            return
        elif modele not in MODELES:
            print('!!! unknow modele:%s !!!' % modele)
            return
        
        old_pron = pron[1]
        new_pron = ''
        if verbose:
            print('old_pron:%s ' % old_pron)
        subfields = old_pron.split('|')
        if verbose:
            print(subfields)
        
        first_untag_met = False
        for subfield in subfields:
            prefix = ''
            if verbose:
                print('subfield:%s' % subfield)
            #print('subfield splitted', subfield.split('='))
            if '=' in subfield:
                subsplits = subfield.split('=')
                subfield=subsplits[1]
                prefix = subsplits[0]+'='
                if modele in MODELES_TROIS and prefix in ['ps=', 'pp=']:
                    check_phonemes(subfield, verbose)
                    subfield = subfield.replace(old_symbol, new_symbol)
                    # take care leaving the loop for fr-accord-cons
                
            else:
                if modele in MODELES_SIMPLES:
                    check_phonemes(subfield, verbose)
                    subfield = subfield.replace(old_symbol, new_symbol)
                    # take care leaving the loop for fr-accord-cons
                elif modele in MODELES_DOUBLES:
                    if first_untag_met == False:
                        first_untag_met = True
                    else:
                        check_phonemes(subfield, verbose)
                        subfield = subfield.replace(old_symbol, new_symbol)
            new_pron += prefix + subfield + '|'
        
        # remove last '|'
        new_pron = new_pron[:-1]
        if verbose:
            print('new_pron:', new_pron)
        
        if new_pron != old_pron:
            new_string = pron[0] + new_pron + pron[2]
            print('replacing: %s\nwith     : %s' % (old_string, new_string))    
            text = text.replace(old_string, new_string)
            nb_replacements += 1
        else:
            print('no need to replace: %s' % (old_string))    
            

    print('nb_replacements in model:%d', nb_replacements)
    return text

In [11]:
def process_words(words, old_symbol, new_symbol, commit=False, verbose=False):
    
    # trick to better identify the new moifications
    if commit == False:
        if not '*' in new_symbol:
            new_symbol = '*' + new_symbol + '*'
    
    for word in words:
        
        page = pywikibot.Page(site, word)
        if not page.exists():
            print('page %s does not exists' % word)
            continue
        print('page=%s' % word)
        #print(page.getVersionHistoryTable())
        old_text = page.text
        if verbose:
            print('#########################')
            #print('old page text:', old_text)
            #print('#########################')
            
        if skip_word(old_symbol, page.text, word):
            print()
            continue
        
        
        # Look for the pronunciation within the "ligne de forme"
        pron_head = '{{pron'
        pron_foot = 'fr}}'
        regexp = '('+ pron_head + '\|)'+ '([^\|\}]{1,})' + '(\|'+pron_foot+')'
        pronunciations = re.findall(regexp, page.text)
        new_text = replace_pronunciations_in_ligne_de_forme(page.text, pronunciations, 
                                                            old_symbol, new_symbol, verbose)

        page.text = new_text
        
        # Look for the 2nd pronunciation
        pron_hd = '{{fr'
        pron_ft = '}}'
        regexp = '('+pron_hd + '.*?' + '\|)'+ '([^\}]{1,})' + '(' + pron_ft+')'
        pronunciations = re.findall(regexp, page.text)
        new_text = replace_pronunciations_in_model(page.text, pronunciations, 
                                                   old_symbol, new_symbol, verbose)
        
        if verbose:
            foo = 1
            #print('#########################')
            #print('new page text:', new_text)
            #print('#########################')
            
        if new_text != old_text:
            # string to summarize the work done in the edit comment
            summary = "prononciation: remplacement de /"+ old_symbol + "/ par /"+ new_symbol +"/."
            print('summary: ',summary)
        
            page.text = new_text
            if commit:
                if '*' in new_symbol:
                    print('* in new_symbol %s, probably an error', new_symbol)
                    break
                print('committing')
                page.save(summary=summary, botflag=True, quiet=False)
                # sleep 10 seconds before jumping to the next word
                time.sleep(10)
        else:
            print('no modification')
            
        print('')

In [13]:
process_words(words, old_symbol, new_symbol, commit=True, verbose=False)

page=Aumont
replacing : {{pron|o.mõ|fr}}
with      : {{pron|o.mɔ̃|fr}}
nb_replacements in ldf:%d 1
replacing: {{fr-inv|o.mõ|inv_titre=Nom propre}}
with     : {{fr-inv|o.mɔ̃|inv_titre=Nom propre}}
nb_replacements in model:%d 1
summary:  prononciation: remplacement de /õ/ par /ɔ̃/.
committing


Page [[wiktionary:fr:Aumont]] saved



page=Avignon-lès-Saint-Claude
replacing : {{pron|a.vi.ɲõ.lɛ.sɛ̃.klod|fr}}
with      : {{pron|a.vi.ɲɔ̃.lɛ.sɛ̃.klod|fr}}
nb_replacements in ldf:%d 1
replacing: {{fr-inv|a.vi.ɲõ.lɛ.sɛ̃.klod|inv_titre=Nom propre}}
with     : {{fr-inv|a.vi.ɲɔ̃.lɛ.sɛ̃.klod|inv_titre=Nom propre}}
nb_replacements in model:%d 1
summary:  prononciation: remplacement de /õ/ par /ɔ̃/.
committing


Page [[wiktionary:fr:Avignon-lès-Saint-Claude]] saved



page=Caf’-Con’
replacing : {{pron|kaf.kõ|fr}}
with      : {{pron|kaf.kɔ̃|fr}}
nb_replacements in ldf:%d 1
replacing: {{fr-inv|kaf.kõ}}
with     : {{fr-inv|kaf.kɔ̃}}
nb_replacements in model:%d 1
summary:  prononciation: remplacement de /õ/ par /ɔ̃/.
committing


...

Page [[wiktionary:fr:union libre]] saved



page=volontourisme
replacing : {{pron|vɔ.lõ.tu.ʁism|fr}}
with      : {{pron|vɔ.lɔ̃.tu.ʁism|fr}}
nb_replacements in ldf:%d 1
replacing: {{fr-rég|vɔ.lõ.tu.ʁism}}
with     : {{fr-rég|vɔ.lɔ̃.tu.ʁism}}
nb_replacements in model:%d 1
summary:  prononciation: remplacement de /õ/ par /ɔ̃/.
committing


Page [[wiktionary:fr:volontourisme]] saved



page=volontourismes
replacing : {{pron|vɔ.lõ.tu.ʁism|fr}}
with      : {{pron|vɔ.lɔ̃.tu.ʁism|fr}}
nb_replacements in ldf:%d 1
replacing: {{fr-rég|vɔ.lõ.tu.ʁism|s=volontourisme}}
with     : {{fr-rég|vɔ.lɔ̃.tu.ʁism|s=volontourisme}}
nb_replacements in model:%d 1
summary:  prononciation: remplacement de /õ/ par /ɔ̃/.
committing


Page [[wiktionary:fr:volontourismes]] saved
